In [1]:
//init cell

//common functions

const os = require('os');
const fs = require('fs');
const osc = require('osc');
const net = require('net');

function normalizeTime(loop) {
    let touch_pts = loop.slice(1, -1);
    const start_time = touch_pts[0].ts;
    const dur = touch_pts.slice(-1)[0].ts - start_time;
    touch_pts = touch_pts.map(({ts, pos}) => ({pos, ts: (ts-start_time)/dur}));
    return touch_pts;
}

function deduplicate(loop) { //for fixing touchOSC bug of "lagging Y" redundant pre-message
    let filteredLoop = loop.filter((elem, i, arr) => i%2 == 0);
    return filteredLoop;
    //todo - this only works when messages are guaranteed ordered (local UDP, tcp)
    //for non-ordered messages, probably most robust is to just toss first couple values to
    //get rid of "cut start" from redundant message (or to move cursor near intened start before recording)
}

function oscArgs(...args) {
    let output  = args.map(e => {
        let type_str = typeof e;
        switch(type_str) {
            case 'string':
                return {type: 's', value: e}
            case 'number': 
                return {type: 'f', value: e}
            case 'boolean':
                return {type: 'i', value: e+0}
        }
    });
    return output;
}

function saveLoops(fileName) {
    if(!fileName) fileName = Date().toString().replaceAll(/[\: \)\(]/g, "-") + ".json";
    fs.writeFile(fileName, JSON.stringify({rawLoops, processedLoops}));
}
function loadLoops(fileName) {
    return JSON.parse(fs.readFileSync(fileName));
}

//TCP server to OF

const tcp_port = 11999;
const tcp_host = '127.0.0.1';

const server = net.createServer();
server.listen(tcp_port, tcp_host, () => {
    console.log('TCP Server is running on port ' + tcp_port + '.');
});

const sockets = [];

server.on('connection', function(sock) {
    console.log('CONNECTED: ' + sock.remoteAddress + ':' + sock.remotePort);
    sockets.push(sock);

    sock.write( JSON.stringify(processedLoops) + '\n' );

    sock.on('data', function(data) {
        console.log('DATA ' + sock.remoteAddress + ': ' + data);
        // Write the data back to all the connected, the client will receive it as data from the server
        sockets.forEach(function(sock, index, array) {
            sock.write(sock.remoteAddress + ':' + sock.remotePort + " said " + data + '\n');
        });
    });

    // Add a 'close' event handler to this instance of socket
    sock.on('close', function(data) {
        let index = sockets.findIndex(function(o) {
            return o.remoteAddress === sock.remoteAddress && o.remotePort === sock.remotePort;
        })
        if (index !== -1) sockets.splice(index, 1);
        console.log('CLOSED: ' + sock.remoteAddress + ' ' + sock.remotePort);
    });
});

//OSC Server to touchOSC

const TCP_OSC_PORT = 7071;
const TCP_OSC_HOST = '192.168.1.39';

const tcp_osc_port = new osc.TCPSocketPort({});
// change to remote host/port (address of phone/tablet) - set touchOSC on device to TCPserver
tcp_osc_port.open(TCP_OSC_HOST, TCP_OSC_PORT); 

tcp_osc_port.on('ready', () => {
  console.log('ready TCP');
});


const UDP_OSC_PORT = 7071;
const UDP_OSC_HOST = '0.0.0.0';

// Bind to a UDP socket to listen for incoming OSC events.
const udp_osc_port = new osc.UDPPort({
    localAddress: UDP_OSC_HOST,
    localPort: UDP_OSC_PORT,
    remoteAddress: '127.0.0.1',
    remotePort: 7072
});

udp_osc_port.on("ready", () => {
    console.log('ready UDP');
});

udp_osc_port.open();

let osc_port = udp_osc_port;

let rawLoops = {};
let processedLoops = {};
let activeLoop = 'firstLoop';
let isRecording = false;
let isTouching = false;
const handlers = {};

function processLoop(loopKey) {
    processedLoops[loopKey] = deduplicate(normalizeTime(rawLoops[loopKey]));
}

handlers['/recording'] = (isRec) => {
    isRecording = isRec;
    if(isRecording) {
        rawLoops[activeLoop] = [{ts: Date.now(), pos: 'start'}];
    } else {
        rawLoops[activeLoop].push({ts: Date.now(), pos: 'end'});
        processLoop(activeLoop);
        sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ))
    }
}
const f = float => ({type: 'f', value: float}); //helper for formatting floats for OSC
handlers['/pos'] = (x, y) => {
    if(isRecording) {
        rawLoops[activeLoop].push({ts: Date.now(), pos: {x, y}});
    }
    udp_osc_port.send({address: '/touchPos', args: [f(x), f(y)]});
}
handlers['/pos/touch'] = (touching) => {
    isTouching = touching;
    udp_osc_port.send({address: '/touching', args: [f(isTouching)]});
}

let logAddresses = [];
osc_port.on('message', (message) => {
    let {address, args} = message;
    if(logAddresses.includes(address)) console.log(message);
    if(handlers[address]) handlers[address](...args); 
});

let loadFile = "test_loops.json";
if(loadFile) {
    console.log("loaded file", loadFile);
    let loops = loadLoops(loadFile);
    processedLoops = loops.processedLoops;
    rawLoops = loops.rawLoops;
}

"Setup Done"

loaded file test_loops.json


'Setup Done'

ready UDP
TCP Server is running on port 11999.
CONNECTED: 127.0.0.1:59728


In [2]:
activeLoop = "loop3"

'loop3'

In [ ]:
sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ))

In [ ]:
udp_osc_port.send({address: "/launch", args: oscArgs('firstLoop', 3, false, 'grp1', 'key1')});

In [4]:
for(var i = 0; i < 100; i++) {
    let dur = 5 + i/10
    udp_osc_port.send({address: "/launch", args: oscArgs('loop3', dur, false, 'grp1', 'key1')});
}

CLOSED: 127.0.0.1 59728


In [ ]:
function deduplicate(loop) { //for fixing touchOSC bug of "lagging Y" redundant pre-message
    let filteredLoop = loop.filter((elem, i, arr) => i%2 == 0);
    return filteredLoop;
    //todo - this only works when messages are guaranteed ordered (local UDP, tcp)
    //for non-ordered messages, probably most robust is to just toss first couple values to
    //get rid of "cut start" from redundant message (or to move cursor near intened start before recording)
}

In [ ]:
deduplicate(rawLoops['firstLoop']).length / rawLoops['firstLoop'].length

In [ ]:
processedLoops['firstLoop'].map(e => e.ts)